In [1]:
import numpy as np

In [35]:
x = np.random.randint(0, 10, size=[4, 4, 3])
y = np.random.randint(0, 10, size=[4, 4, 3])
mask = np.full_like(x, fill_value=False)

In [44]:
masked_mae(x, y, mask=mask)

3.75

In [51]:
masked_mae(x, y, mask=mask, axis=(0, 1))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


array([ nan, 3.75,  nan])

In [42]:
masked_mae(x, y)

3.0208333333333335